# Exploring transformers architecture

In [2]:
import os
import sys
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from torch import nn

print(torch.__version__)

notebook_dir = os.getcwd()

project_root = os.path.abspath(os.path.join(notebook_dir, "../.."))
print(project_root)
if project_root not in sys.path:
    sys.path.append(project_root)

2.2.2
/Users/damianstone/Documents/Code/machine-learning/dl-sepsis-prediction


## Dataset

In [ ]:
from utils import get_data

DATA_PATH = get_data.get_dataset_abspath()
load_path = os.path.join(DATA_PATH, "imputed_combined_data.parquet")
imputed_df = pd.read_parquet(load_path)

In [15]:
imputed_df.head(1)

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,HCO3,FiO2,...,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel,patient_id,dataset,cluster_id
0,102.108491,91.419811,36.789519,128.165094,88.199717,71.73211,24.712264,-0.288406,23.835971,0.467029,...,83.14,0,0.715787,0.284213,-0.03,1,0,1,A,0_9_2_2_X_X_X


In [17]:
if 'SOFA' in imputed_df.columns:
    print("Column exists.")
else:
    print("Column doesn't exist.")
    
imputed_df.columns

Column doesn't exist.


Index(['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'BaseExcess',
       'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN', 'Alkalinephos',
       'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct', 'Glucose',
       'Lactate', 'Magnesium', 'Phosphate', 'Potassium', 'Bilirubin_total',
       'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC', 'Fibrinogen', 'Platelets',
       'Age', 'Gender', 'Unit1', 'Unit2', 'HospAdmTime', 'ICULOS',
       'SepsisLabel', 'patient_id', 'dataset', 'cluster_id'],
      dtype='object')

## Last feature engineering

In [18]:
# SOFA calculation based on Sepsis-3
def calculate_sofa(row):
    sofa = 0
    
    if row['FiO2'] > 0:
        pao2_fio2 = row['SaO2'] / row['FiO2']
        if pao2_fio2 < 100: sofa += 4
        elif pao2_fio2 < 200: sofa += 3
        elif pao2_fio2 < 300: sofa += 2
        elif pao2_fio2 < 400: sofa += 1

    if row['Platelets'] < 20: sofa += 4
    elif row['Platelets'] < 50: sofa += 3
    elif row['Platelets'] < 100: sofa += 2
    elif row['Platelets'] < 150: sofa += 1

    if row['Bilirubin_total'] >= 12: sofa += 4
    elif row['Bilirubin_total'] >= 6: sofa += 3
    elif row['Bilirubin_total'] >= 2: sofa += 2
    elif row['Bilirubin_total'] >= 1.2: sofa += 1

    if row['MAP'] < 70: sofa += 1

    if row['Creatinine'] >= 5: sofa += 4
    elif row['Creatinine'] >= 3.5: sofa += 3
    elif row['Creatinine'] >= 2: sofa += 2
    elif row['Creatinine'] >= 1.2: sofa += 1

    return sofa

imputed_df = imputed_df.drop(columns=["Unit1", "Unit2", "cluster_id","dataset", "patient_id"], errors='ignore')
imputed_df["SOFA"] = imputed_df.apply(calculate_sofa, axis=1)
imputed_df.head(1)


,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,HCO3,FiO2,...,PTT,WBC,Fibrinogen,Platelets,Age,Gender,HospAdmTime,ICULOS,SepsisLabel,SOFA
0,102.108491,91.419811,36.789519,128.165094,88.199717,71.73211,24.712264,-0.288406,23.835971,0.467029,...,36.405357,11.737903,350.0,224.187135,83.14,0,-0.03,1,0,2


In [ ]:
imputed_df.to_parquet(f"{project_root}/dataset/imputed_sofa.parquet", index=False)

## Splitting the data

In [3]:
from utils import get_data

DATA_PATH = get_data.get_dataset_abspath()
load_path = os.path.join(DATA_PATH, "imputed_sofa_ds.csv")
imputed_df = pd.read_csv(load_path)

In [4]:
from sklearn.model_selection import train_test_split

X = imputed_df.drop(columns=['SepsisLabel'])
y = imputed_df['SepsisLabel']

# dataframe to numpy
X_np = X.values
y_np = y.values 

X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.2, random_state=42)

# numpy to tensors
X_train= torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [3]:
from get_splitted_data import get_dataset_tensors

X_train, X_test, y_train, y_test = get_dataset_tensors()

from saved dataset


## Transformer architecture
- Use `Mean Pooling` (x.mean(dim=1)) if: You want a summary of the whole sequence (useful if sepsis patterns are spread across time).
- Use `Max Pooling` (x.max(dim=1).values) if: You want to focus on the most extreme feature values, which might indicate critical moments.
- Use `Last Timestep` (x[:, -1, :]) if: You believe the latest patient state matters most (recommended for your case).

Test pooling:
- Train with different pooling methods and compare AUC-ROC scores.
- If AUC increases, the new pooling method is better.
- Visualize attention weights to see where the model is focusing.

In [12]:
class TransformerClassifier(nn.Module):
    """
    num_heads = more heads capture different attention but increase computation
    num_layers = more make the model deeper but can overfit if too high
    """
    def __init__(self, input_dim, num_heads=4, num_layers=2):
        super().__init__()
        # d_model = input_dim (number of features)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads)
        # stacks multiple encoder layers (num_layers controls depth)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        # linear layer to map the output to a single value (binary classification)
        self.linear_layer = nn.Linear(in_features=input_dim, out_features=1)

    def forward(self, x):
      z = self.encoder(x)  # Output shape: (batch_size, seq_len, features) or (batch_size, features)
      if z.dim() == 3:  # If (batch_size, seq_len, features), take the last timestep
            # NOTE: last timestep -> the most recent ICU data is usually the most relevant for prediction
            z = z[:, -1, :]
      return torch.sigmoid(self.linear_layer(z))


def get_valid_num_heads(input_dim, desired_heads):
    """Finds the highest valid num_heads <= desired_heads that divides input_dim."""
    while desired_heads > 0 and input_dim % desired_heads != 0:
        desired_heads -= 1
    return max(1, desired_heads)


in_dim = X_train.shape[1]
print(in_dim)
n_heads = get_valid_num_heads(in_dim, 10)
print(n_heads)
model = TransformerClassifier(input_dim=in_dim, num_heads=n_heads)

38
2


/Users/damianstone/Documents/Code/machine-learning/dl-sepsis-prediction/venv-sepsis/lib/python3.12/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [13]:
batch_size = 512
y_pred_list = []
with torch.inference_mode():
    for i in range(0, len(X_test), batch_size):
        batch = X_test[i:i+batch_size]
        y_pred_list.append(model(batch))
y_pred = torch.cat(y_pred_list)
y_pred[:5]

tensor([[0.7244],
        [0.7459],
        [0.7783],
        [0.7931],
        [0.7251]])